# How to get reverse kinetics for a rxn?

This ipynb will solve the commonly seen question, what is the reverse reaction rate at certain condition given the forward (A, n, Ea) kinetics.

- input: reactants and products of forward reaction, forward kinetics (A, n, Ea), temperature of interest

- output: reverse reaction rate at input temperature

## Inputs

In [46]:
# reactants and products
reactants = ['C=Cc1ccccc1', 'CCCCCCCCC[CH2]']
products = ['CCCCCCCCCCC[CH]c1ccccc1']

In [47]:
# forward kinetics (A, n, Ea)
A = 7.820e+02 # cm3/mol/s
n = 2.41
Ea = 3.71 # kcal/mol

In [48]:
# temperature of interest
T = 673 # K

## Calculation procedure

In [49]:
from rmgpy.rmg.model import Species
from rmgpy.data.thermo import ThermoDatabase, NASA
import math
R = 8.314
thermodb = ThermoDatabase()
path = '/home/kehang/Code/rmgmaster/RMG-database/input/thermo'
thermodb.load(path)

In [50]:
# get Gibbs free energy for all the species
rxts_prds_spc = []
for smiles in (reactants + products):
    spc = Species(label=smiles).fromSMILES(smiles)
    spc.generateResonanceIsomers() # really important!
    rxts_prds_spc.append(spc)
    # calculate thermo
    thermo = thermodb.getThermoData(spc, None, None)
    _ = spc.processThermoData(None, thermo, NASA)
    

In [51]:
# get deltaG for the forward reaction
deltaG = 0
for spc in rxts_prds_spc:
    if spc.label in reactants:
        deltaG -= spc.getFreeEnergy(T)
        print spc
        print spc.getEnthalpy(T)/4184
    elif spc.label in products:
        deltaG += spc.getFreeEnergy(T) 
        print spc
        print spc.getEnthalpy(T)/4184

C=Cc1ccccc1
51.9260695668
CCCCCCCCC[CH2]
19.1663979162
CCCCCCCCCCC[CH]c1ccccc1
38.8364744444


In [52]:
# calculate equilibrium const K
order = len(products)-len(reactants)
print order
K = math.exp(-deltaG/R/T)*(1/(R*T)/10)**order
K

-1


5113481.621509884

In [53]:
# calculate forward k_f
k_f = A*(T**n)*math.exp(-Ea*4184/R/T)
print k_f

319077163.267


In [54]:
# calculate reverse k_r by k_f/K
k_r = k_f/K
print k_r

62.3992001701


Next steps:

- the equilibrium const need more investigation when the numbers of products and reactants are not equal